In [5]:
import cv2
import numpy as np
import random

# Function to detect the number of fingers raised
def count_fingers(roi):
    # Convert ROI to grayscale
    gray = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
    blurred = cv2.GaussianBlur(gray, (35, 35), 0)
    _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

    # Find contours
    contours, _ = cv2.findContours(thresh.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    if not contours:
        return 0

    # Find the largest contour
    cnt = max(contours, key=lambda x: cv2.contourArea(x))
    hull = cv2.convexHull(cnt)

    # Find convexity defects
    hull_indices = cv2.convexHull(cnt, returnPoints=False)
    defects = cv2.convexityDefects(cnt, hull_indices)

    if defects is None:
        return 0

    finger_count = 0
    for i in range(defects.shape[0]):
        s, e, f, d = defects[i, 0]
        start = tuple(cnt[s][0])
        end = tuple(cnt[e][0])
        far = tuple(cnt[f][0])

        # Calculate the distance from start to end
        a = np.linalg.norm(np.array(start) - np.array(end))
        b = np.linalg.norm(np.array(start) - np.array(far))
        c = np.linalg.norm(np.array(end) - np.array(far))

        # Calculate the angle using cosine rule
        angle = np.arccos((b**2 + c**2 - a**2) / (2 * b * c)) * 57

        # If angle is less than 90 degrees, we consider it as a finger
        if angle <= 90:
            finger_count += 1

    # We add 1 for the thumb, if it is visible
    return min(finger_count + 1, 6)  # Limit to 6 fingers (hand cricket rule)

# Game logic for Hand Cricket
def hand_cricket():
    cap = cv2.VideoCapture(0)
    user_score = 0
    computer_score = 0
    is_user_out = False

    # User bowls first
    print("You will bowl first!")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # Flip the frame for a mirror effect
        roi = frame[100:400, 100:400]  # Define the region of interest for hand detection

        # Display prompt to press "c" key to capture input
        cv2.putText(frame, 'Press "c" to bowl!', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw the region of interest (ROI) for hand gesture detection
        cv2.rectangle(frame, (100, 100), (400, 400), (255, 0, 0), 2)

        # Wait for the player to press the "c" key to capture input
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            user_choice = count_fingers(roi)  # Count the number of fingers shown by the player
            
            # Computer randomly chooses a number to bat
            computer_choice = random.randint(1, 6)
            
            # Display player and computer choices on the screen
            cv2.putText(frame, f'Your Bowl: {user_choice}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f'Computer Batted: {computer_choice}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Check if the computer is out (choices match)
            if user_choice == computer_choice:
                is_user_out = True
                break
            
            # Update computer's score
            if computer_choice != 0:
                computer_score += computer_choice
        
        # Display current score
        cv2.putText(frame, f'Computer Score: {computer_score}', (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        # Show the updated frame
        cv2.imshow('Hand Cricket', frame)

        # Exit the game if the "q" key is pressed
        if key == ord('q'):
            break

    # If user is out, display game over message
    cap.release()
    cv2.destroyAllWindows()

    # Display game over for the user and final score of the computer
    print(f"Game Over! Computer Score: {computer_score}")
    if is_user_out:
        print("The computer is out!")
    else:
        print("You ended the game voluntarily.")

    # Now, user bats while computer bowls
    print("\nNow it's your turn to bat!")
    cap = cv2.VideoCapture(0)
    is_computer_out = False

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        frame = cv2.flip(frame, 1)  # Flip the frame for a mirror effect
        roi = frame[100:400, 100:400]  # Define the region of interest for hand detection

        # Display prompt to press "c" key to capture input
        cv2.putText(frame, 'Press "c" to bat!', (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Draw the region of interest (ROI) for hand gesture detection
        cv2.rectangle(frame, (100, 100), (400, 400), (255, 0, 0), 2)

        # Wait for the player to press the "c" key to capture input
        key = cv2.waitKey(1) & 0xFF
        if key == ord('c'):
            player_choice = count_fingers(roi)  # Count the number of fingers shown by the player
            
            # Computer randomly chooses a number to bowl
            computer_choice = random.randint(1, 6)
            
            # Display player and computer choices on the screen
            cv2.putText(frame, f'Your Bat: {player_choice}', (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(frame, f'Computer Bowled: {computer_choice}', (10, 150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            # Check if the user is out (choices match)
            if player_choice == computer_choice:
                is_computer_out = True
                break
            
            # Update user's score
            if player_choice != 0:
                user_score += player_choice
        
        # Display current score
        cv2.putText(frame, f'Your Score: {user_score}', (10, 200), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        # Show the updated frame
        cv2.imshow('Hand Cricket', frame)

        # Exit the game if the "q" key is pressed
        if key == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

    # Display game over for the computer and final score of the user
    print(f"Game Over! Your Score: {user_score}")
    if is_computer_out:
        print("You are out!")
    else:
        print("You ended the game voluntarily.")

    # Compare scores to determine the winner
    print("\nFinal Scores:")
    print(f"User Score: {user_score}")
    print(f"Computer Score: {computer_score}")
    
    if user_score > computer_score:
        print("Congratulations! You win!")
    elif user_score < computer_score:
        print("The computer wins! Better luck next time.")
    else:
        print("It's a tie!")

# Run the hand cricket game
if __name__ == '__main__':
    hand_cricket()
